# Application for Analysis of Fire in Victorian Cities

Date started: 29/04/2019 <br>
Last modified: 24/05/2019

Version: 1.0

Environment: Python 3.6.0 and Anaconda 4.3.0 (64-bit)

Libraries used:
* pandas 
* pymongo
* datetime
* json
* pprint


# 1.INTRODUCTION
StopFire is a campaign started by Monash University to predict and stop the fire in Victorian cities. They have employed sensors in different cities of Victoria and have collected a large amount of data.
This application migrates their data to NoSQL database (MongoDB), analyzes their data and provides them with the results in the form of visualizations. It is a complete application from streaming to storing and analyzing the data for them using Apache Kafka, Apache Spark Streaming and MongoDB.

This assessment includes five csv files such as hotspot_historic.csv,climate_historic.csv, hotspot_AQUA_streaming.csv, hotspot_TERRA_streaming.csv and climate_streaming.csv.

Each datasets involves certain traits needed to predict and stop the fire in Victorian cities.  

# TASK A. 
## A.1. MONGODB Data Model
Referential Data Model was chosen.The considered datasets for this task are climate_historic.csv and hotspot_historic.csv. <br>

The climate_historic data consists of 7 columns or keys such as 
* climate_id 
* station_ID
* air_temperature_celcius
* relative_humidity
* windspeed_knots
* max_wind_speed
* precipitation<br>
climate_id column acts as the primary key as the records of the climate data are provided for each day in a certain period. There are no two records with the same date.<br>

The hotspot_historic data consists of 6 columns or keys such as 
* hotspot_id 
* latitude
* longitude
* datetime
* confidence
* surface_temperature_celcius<br>
The hotspot_id contains the dates where fire occured. It is not a unique column because there is a possibility of zero or more fire occurences for any particular day. 

#### JUSTIFICATION:
The hotspot_historic dataset has records of fire occurrences happening zero or more times with respect to climate_historic data's date. There might be zero or more fire occurrence per day in climate_historic data.
The Referential data model is chosen as it can be used to easily share, categorise and describe the data better.  


# TASK B. 

## TASK B.1.

### Import Libraries

In [1]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import json
from pprint import pprint
import matplotlib.pyplot as plt

### Loading of input data CSV files

In [2]:
climate_data=pd.read_csv("/home/pradnya/monash/5148/Assignment/Assignment Data-20190504/climate_historic.csv")
hotspot_data=pd.read_csv("/home/pradnya/monash/5148/Assignment/Assignment Data-20190504/hotspot_historic.csv")

In [3]:
client = MongoClient ()

### Creation of connection with MongoDB

In [4]:
db=client.fit5148_assignment_db

### Accessing Collection

In [5]:
climate=db.climate

In [6]:
hotspot=db.hotspot

### Creation of climateHistoric Json variable

In [7]:
climate_json=[]
for index,row in climate_data.iterrows():
    climate_json.append(
        {
            "climate_id": row["date"],
            "station_ID": row["station"],
            "air_temperature_celcius": row["air_temperature_celcius"],
            "relative_humidity": row["relative_humidity"],
            "windspeed_knots": row["windspeed_knots"],
            "max_wind_speed": row["max_wind_speed"],
            "precipitation": row['precipitation ']
    
    })

### Creation of HotspotHistoric Json variable

In [8]:
hotspot_json=[]
for index,row in hotspot_data.iterrows():
    hotspot_json.append(
        {
            "hotspot_id":row["date"],
            "latitude": row["latitude"],
            "longitude": row['longitude'],
            "datetime": str(row['datetime']),
            "confidence" : row['confidence'],
            "surface_temperature_celcius": row['surface_temperature_celcius']
        })
            
            

### Insert multiple documents to collection

In [9]:
climate_coll= climate.insert_many(climate_json)

In [10]:
hotspot_coll= hotspot.insert_many(hotspot_json)

## TASK B 2.

### TASK B 2.a- The Climate Data on 10/12/2017.

The climate_id  acts as the date column in the climate data documents and all the measures recorded on 10/12/2017 are displayed with the help of find() function. 

In [11]:

date_val=climate.find({"climate_id":"10/12/2017"})
for val in date_val:
    pprint(val)

{'_id': ObjectId('5dcd3962201a47a71577bb3a'),
 'air_temperature_celcius': 17,
 'climate_id': '10/12/2017',
 'max_wind_speed': 14.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 53.5,
 'station_ID': 948702,
 'windspeed_knots': 7.3}


### TASK B 2.b.- When the surface_temperature_celcius is between 65(​ °C) and 100(​ °C) , the Latitude, Longitude, surface_temperature_celcius and confidence are:

The hotspot data documents consists records of the location of the fire occurrence, surface_temperature in celciusand confidence rate when the surface_temperature is between 65(​ °C) and 100(​ °C) using "$and" operator.

In [12]:

vals_hotspot=hotspot.find({"$and": [{"surface_temperature_celcius":{"$gte": 65}}, {"surface_temperature_celcius":{"$lte": 100}}]})
list_keys=["latitude","longitude","surface_temperature_celcius","Confidence"]
for each in vals_hotspot:
    list_vals=[each['latitude'],each['longitude'],each['surface_temperature_celcius'],each['confidence']]
    dict_hotspot=dict(zip(list_keys,list_vals))
    print("\n",dict_hotspot)


 {'latitude': -37.966, 'longitude': 145.05100000000002, 'surface_temperature_celcius': 68, 'Confidence': 78}

 {'latitude': -35.543, 'longitude': 143.316, 'surface_temperature_celcius': 67, 'Confidence': 86}

 {'latitude': -37.875, 'longitude': 142.51, 'surface_temperature_celcius': 73, 'Confidence': 93}

 {'latitude': -37.613, 'longitude': 149.305, 'surface_temperature_celcius': 75, 'Confidence': 95}

 {'latitude': -37.624, 'longitude': 149.314, 'surface_temperature_celcius': 66, 'Confidence': 90}

 {'latitude': -38.056999999999995, 'longitude': 144.211, 'surface_temperature_celcius': 73, 'Confidence': 93}

 {'latitude': -37.95, 'longitude': 142.366, 'surface_temperature_celcius': 70, 'Confidence': 92}

 {'latitude': -36.282, 'longitude': 146.157, 'surface_temperature_celcius': 71, 'Confidence': 100}

 {'latitude': -37.641999999999996, 'longitude': 149.263, 'surface_temperature_celcius': 65, 'Confidence': 100}

 {'latitude': -37.634, 'longitude': 149.237, 'surface_temperature_celcius

### TASK B 2.c- On 15/12/2017 and 16/12/2017, the surface_temperature in celcius, air temperature in celcius, relative humidity and max wind speed are 

This question involves aggregation of two data collections (climate and hotspot). The "match" opeartor is used to choose the documents for 15 and 16/12/2017 dates. The "lookup" operator is used for merging of two collections and finding the  records of the matched values from the common column. Here, the date columns such as _id from climate  (localfield)  and hotspot_id  from hotspot  (foreignfield) data are the common ones and the surface temperature values are chosen (as field)


In [13]:

date_match = climate.aggregate([
    {"$match":{"climate_id": {"$in":["15/12/2017","16/12/2017"]}}},
    {
"$lookup":
    {
        "from": "hotspot", 
        "localField": "climate_id",
        "foreignField" : "hotspot_id",
        "as": "surface_temperature_celcius"
    }
    },
    {
        "$unwind":"$surface_temperature_celcius"
    },
    {
    "$project": {
        "air_temperature_celcius": 1,
     "max_wind_speed": 1,
     "relative_humidity": 1,
    "surface_temperature_celcius.surface_temperature_celcius": 1
    }
}
])
for match in date_match:
    pprint(match)

   

{'_id': ObjectId('5dcd3962201a47a71577bb3f'),
 'air_temperature_celcius': 18,
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celcius': {'surface_temperature_celcius': 42}}
{'_id': ObjectId('5dcd3962201a47a71577bb3f'),
 'air_temperature_celcius': 18,
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celcius': {'surface_temperature_celcius': 36}}
{'_id': ObjectId('5dcd3962201a47a71577bb3f'),
 'air_temperature_celcius': 18,
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celcius': {'surface_temperature_celcius': 38}}
{'_id': ObjectId('5dcd3962201a47a71577bb3f'),
 'air_temperature_celcius': 18,
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0,
 'surface_temperature_celcius': {'surface_temperature_celcius': 40}}
{'_id': ObjectId('5dcd3962201a47a71577bb40'),
 'air_temperature_celcius': 18,
 'max_wind_speed': 13.0,
 'relative_humidity': 53.7,
 'surface_temperature_celcius': {'surface_temperature_celcius': 43}}


### TASK B 2.d- When the confidence is between 80 and 100, the datetime, air temperature (°C), surface temperature (°C) are :

The range of confidence values are matched using "$gte" and "$lte" operators. The "lookup" operator is used for merging of two collections and finding the  records of the matched values from the common column. Here, the date columns such as _id from climate (foreignfield)  and hotspot_id  from hotspot   (localfield)  data are the common ones and the air temperature values are chosen (as field)

In [14]:


datetime_match =  hotspot.aggregate([
    {"$match":{"$and": [{"confidence":{"$gte": 80}}, {"confidence":{"$lte": 100}}]}},
    {
"$lookup":
    {
        "from": "climate",
        "localField": "hotspot_id",
        "foreignField" : "climate_id",
        "as": "air_temperature_celcius"
    },
    
    }])
for match in datetime_match:
    pprint(match)
   

{'_id': ObjectId('5dcd3964201a47a71577bb52'),
 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577bb4b'),
                              'air_temperature_celcius': 28,
                              'climate_id': '27/12/2017',
                              'max_wind_speed': 15.9,
                              'precipitation': ' 0.00I',
                              'relative_humidity': 58.3,
                              'station_ID': 948702,
                              'windspeed_knots': 9.3}],
 'confidence': 82,
 'datetime': '2017-12-27T00:02:15',
 'hotspot_id': '27/12/2017',
 'latitude': -35.541,
 'longitude': 143.311,
 'surface_temperature_celcius': 63}
{'_id': ObjectId('5dcd3964201a47a71577bb54'),
 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577bb4b'),
                              'air_temperature_celcius': 28,
                              'climate_id': '27/12/2017',
                              'max_wind_speed': 15.9,
                         

 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577bb1d'),
                              'air_temperature_celcius': 18,
                              'climate_id': '11/11/2017',
                              'max_wind_speed': 12.0,
                              'precipitation': ' 0.00I',
                              'relative_humidity': 55.6,
                              'station_ID': 948702,
                              'windspeed_knots': 8.2}],
 'confidence': 86,
 'datetime': '2017-11-11T04:04:25',
 'hotspot_id': '11/11/2017',
 'latitude': -37.863,
 'longitude': 144.17,
 'surface_temperature_celcius': 60}
{'_id': ObjectId('5dcd3964201a47a71577bbbd'),
 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577bb1b'),
                              'air_temperature_celcius': 14,
                              'climate_id': '9/11/2017',
                              'max_wind_speed': 13.0,
                              'precipitation': ' 0.00I',
                

{'_id': ObjectId('5dcd3964201a47a71577bc26'),
 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577baed'),
                              'air_temperature_celcius': 14,
                              'climate_id': '24/09/2017',
                              'max_wind_speed': 23.9,
                              'precipitation': ' 0.16G',
                              'relative_humidity': 44.0,
                              'station_ID': 948702,
                              'windspeed_knots': 14.4}],
 'confidence': 100,
 'datetime': '2017-09-24T15:07:47',
 'hotspot_id': '24/09/2017',
 'latitude': -37.446,
 'longitude': 148.102,
 'surface_temperature_celcius': 65}
{'_id': ObjectId('5dcd3964201a47a71577bc29'),
 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577baed'),
                              'air_temperature_celcius': 14,
                              'climate_id': '24/09/2017',
                              'max_wind_speed': 23.9,
                       

{'_id': ObjectId('5dcd3964201a47a71577bca0'),
 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577ba87'),
                              'air_temperature_celcius': 11,
                              'climate_id': '14/06/2017',
                              'max_wind_speed': 8.9,
                              'precipitation': ' 0.00I',
                              'relative_humidity': 46.4,
                              'station_ID': 948701,
                              'windspeed_knots': 6.5}],
 'confidence': 94,
 'datetime': '2017-06-14T04:43:00',
 'hotspot_id': '14/06/2017',
 'latitude': -36.2693,
 'longitude': 143.1768,
 'surface_temperature_celcius': 85}
{'_id': ObjectId('5dcd3964201a47a71577bca7'),
 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577ba82'),
                              'air_temperature_celcius': 10,
                              'climate_id': '9/06/2017',
                              'max_wind_speed': 8.9,
                          

{'_id': ObjectId('5dcd3964201a47a71577bd38'),
 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577ba69'),
                              'air_temperature_celcius': 10,
                              'climate_id': '15/05/2017',
                              'max_wind_speed': 8.9,
                              'precipitation': ' 0.00I',
                              'relative_humidity': 47.0,
                              'station_ID': 948701,
                              'windspeed_knots': 5.2}],
 'confidence': 84,
 'datetime': '2017-05-15T04:26:20',
 'hotspot_id': '15/05/2017',
 'latitude': -36.3156,
 'longitude': 141.4514,
 'surface_temperature_celcius': 57}
{'_id': ObjectId('5dcd3964201a47a71577bd39'),
 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577ba69'),
                              'air_temperature_celcius': 10,
                              'climate_id': '15/05/2017',
                              'max_wind_speed': 8.9,
                         

...

                              'precipitation': ' 0.00I',
                              'relative_humidity': 44.2,
                              'station_ID': 948701,
                              'windspeed_knots': 7.8}],
 'confidence': 85,
 'datetime': '2017-04-06T04:22:10',
 'hotspot_id': '6/04/2017',
 'latitude': -36.3739,
 'longitude': 141.3614,
 'surface_temperature_celcius': 59}
{'_id': ObjectId('5dcd3964201a47a71577c3e6'),
 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577ba42'),
                              'air_temperature_celcius': 19,
                              'climate_id': '6/04/2017',
                              'max_wind_speed': 15.9,
                              'precipitation': ' 0.00I',
                              'relative_humidity': 44.2,
                              'station_ID': 948701,
                              'windspeed_knots': 7.8}],
 'confidence': 90,
 'datetime': '2017-04-06T04:22:10',
 'hotspot_id': '6/04/2017',
 'latitude': -

 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577ba41'),
                              'air_temperature_celcius': 17,
                              'climate_id': '5/04/2017',
                              'max_wind_speed': 13.0,
                              'precipitation': ' 0.00I',
                              'relative_humidity': 47.2,
                              'station_ID': 948701,
                              'windspeed_knots': 4.9}],
 'confidence': 89,
 'datetime': '2017-04-05T03:39:10',
 'hotspot_id': '5/04/2017',
 'latitude': -37.4169,
 'longitude': 145.5926,
 'surface_temperature_celcius': 64}
{'_id': ObjectId('5dcd3964201a47a71577c45e'),
 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577ba40'),
                              'air_temperature_celcius': 16,
                              'climate_id': '4/04/2017',
                              'max_wind_speed': 12.0,
                              'precipitation': ' 0.00I',
               

 'latitude': -37.8064,
 'longitude': 146.0189,
 'surface_temperature_celcius': 45}
{'_id': ObjectId('5dcd3964201a47a71577c4bc'),
 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577ba3f'),
                              'air_temperature_celcius': 15,
                              'climate_id': '3/04/2017',
                              'max_wind_speed': 13.0,
                              'precipitation': ' 0.00I',
                              'relative_humidity': 47.7,
                              'station_ID': 948701,
                              'windspeed_knots': 5.9}],
 'confidence': 87,
 'datetime': '2017-04-03T03:59:10',
 'hotspot_id': '3/04/2017',
 'latitude': -37.3735,
 'longitude': 145.886,
 'surface_temperature_celcius': 96}
{'_id': ObjectId('5dcd3964201a47a71577c4be'),
 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577ba3f'),
                              'air_temperature_celcius': 15,
                              'climate_id': '3/04/2017'

                              'precipitation': ' 0.00G',
                              'relative_humidity': 50.6,
                              'station_ID': 948701,
                              'windspeed_knots': 6.0}],
 'confidence': 83,
 'datetime': '2017-03-28T04:30:50',
 'hotspot_id': '28/03/2017',
 'latitude': -36.811,
 'longitude': 142.2655,
 'surface_temperature_celcius': 56}
{'_id': ObjectId('5dcd3964201a47a71577c52b'),
 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577ba39'),
                              'air_temperature_celcius': 18,
                              'climate_id': '28/03/2017',
                              'max_wind_speed': 13.0,
                              'precipitation': ' 0.00G',
                              'relative_humidity': 50.6,
                              'station_ID': 948701,
                              'windspeed_knots': 6.0}],
 'confidence': 80,
 'datetime': '2017-03-28T04:30:20',
 'hotspot_id': '28/03/2017',
 'latitude':

 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577ba2e'),
                              'air_temperature_celcius': 18,
                              'climate_id': '17/03/2017',
                              'max_wind_speed': 15.0,
                              'precipitation': ' 0.00I',
                              'relative_humidity': 51.1,
                              'station_ID': 948701,
                              'windspeed_knots': 8.1}],
 'confidence': 86,
 'datetime': '2017-03-17T04:45:00',
 'hotspot_id': '17/03/2017',
 'latitude': -36.7128,
 'longitude': 141.3986,
 'surface_temperature_celcius': 60}
{'_id': ObjectId('5dcd3964201a47a71577c59c'),
 'air_temperature_celcius': [{'_id': ObjectId('5dcd3962201a47a71577ba2b'),
                              'air_temperature_celcius': 25,
                              'climate_id': '14/03/2017',
                              'max_wind_speed': 13.0,
                              'precipitation': ' 0.00I',
            

### Task B 2.e-  The top 10 records with highest surfacetemperature is considered by sorting the surface_temperature_celcius in descending order and limit is set to 10

In [15]:

top_10=db.hotspot.aggregate(
   [
     { "$sort" : { 'surface_temperature_celcius' : -1} },
       { "$limit": 10 }
   ])
for one in top_10:
    pprint(one)

{'_id': ObjectId('5dcd3964201a47a71577bff4'),
 'confidence': 100,
 'datetime': '2017-04-18T04:52:00',
 'hotspot_id': '18/04/2017',
 'latitude': -38.1665,
 'longitude': 143.062,
 'surface_temperature_celcius': 124}
{'_id': ObjectId('5dcd3964201a47a71577c49e'),
 'confidence': 100,
 'datetime': '2017-04-04T04:32:50',
 'hotspot_id': '4/04/2017',
 'latitude': -36.343,
 'longitude': 142.1986,
 'surface_temperature_celcius': 123}
{'_id': ObjectId('5dcd3964201a47a71577bf66'),
 'confidence': 100,
 'datetime': '2017-05-01T04:14:20',
 'hotspot_id': '1/05/2017',
 'latitude': -36.9318,
 'longitude': 143.0907,
 'surface_temperature_celcius': 122}
{'_id': ObjectId('5dcd3964201a47a71577c58d'),
 'confidence': 100,
 'datetime': '2017-03-18T03:50:50',
 'hotspot_id': '18/03/2017',
 'latitude': -37.016999999999996,
 'longitude': 148.1297,
 'surface_temperature_celcius': 121}
{'_id': ObjectId('5dcd3964201a47a71577bd78'),
 'confidence': 100,
 'datetime': '2017-05-13T04:40:20',
 'hotspot_id': '13/05/2017',
 '

### Task B 2.f. The number of fire in each day 

In [16]:

fire_dates=hotspot.aggregate( [
   { "$group": 
    { "_id": "$hotspot_id", 'total number of fire': { "$sum": 1 } } 
   }

] )
for date in fire_dates:
    pprint(date)

{'_id': '6/03/2017', 'total number of fire': 2}
{'_id': '8/03/2017', 'total number of fire': 2}
{'_id': '9/03/2017', 'total number of fire': 3}
{'_id': '13/03/2017', 'total number of fire': 2}
{'_id': '14/03/2017', 'total number of fire': 10}
{'_id': '19/03/2017', 'total number of fire': 21}
{'_id': '28/03/2017', 'total number of fire': 54}
{'_id': '29/03/2017', 'total number of fire': 1}
{'_id': '1/04/2017', 'total number of fire': 7}
{'_id': '17/03/2017', 'total number of fire': 6}
{'_id': '4/04/2017', 'total number of fire': 89}
{'_id': '7/04/2017', 'total number of fire': 39}
{'_id': '14/04/2017', 'total number of fire': 18}
{'_id': '15/04/2017', 'total number of fire': 69}
{'_id': '31/03/2017', 'total number of fire': 22}
{'_id': '6/04/2017', 'total number of fire': 118}
{'_id': '16/04/2017', 'total number of fire': 18}
{'_id': '18/03/2017', 'total number of fire': 3}
{'_id': '19/04/2017', 'total number of fire': 50}
{'_id': '26/03/2017', 'total number of fire': 17}
{'_id': '22/04

### Task B 2.g. The average  Surface Temperature(°C) are calculated by referencing to the date in hotspot data

In [17]:

avg_SA=db.hotspot.aggregate(
   [
     {
       "$group":
         {
           "_id": "$hotspot_id",
           "Average Surface Temperature(°C)": { "$avg": "$surface_temperature_celcius" }
         }
     }
   ]
)
for SA in avg_SA:
    pprint(SA)

{'Average Surface Temperature(°C)': 60.5, '_id': '6/03/2017'}
{'Average Surface Temperature(°C)': 51.5, '_id': '8/03/2017'}
{'Average Surface Temperature(°C)': 46.666666666666664, '_id': '9/03/2017'}
{'Average Surface Temperature(°C)': 38.5, '_id': '13/03/2017'}
{'Average Surface Temperature(°C)': 65.6, '_id': '14/03/2017'}
{'Average Surface Temperature(°C)': 65.57142857142857, '_id': '19/03/2017'}
{'Average Surface Temperature(°C)': 60.925925925925924, '_id': '28/03/2017'}
{'Average Surface Temperature(°C)': 51.0, '_id': '29/03/2017'}
{'Average Surface Temperature(°C)': 46.714285714285715, '_id': '1/04/2017'}
{'Average Surface Temperature(°C)': 59.5, '_id': '17/03/2017'}
{'Average Surface Temperature(°C)': 62.57303370786517, '_id': '4/04/2017'}
{'Average Surface Temperature(°C)': 50.69230769230769, '_id': '7/04/2017'}
{'Average Surface Temperature(°C)': 61.94444444444444, '_id': '14/04/2017'}
{'Average Surface Temperature(°C)': 59.57971014492754, '_id': '15/04/2017'}
{'Average Surface